In [16]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Imposta la cartella dove ci sono i tuoi file audio .wav
AUDIO_DIR = "data/test/violino/audio"
OUTPUT_NAME = "violino_test.csv"
LABEL = "violino"

# Funzione per estrarre features da un file
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None, duration=10)

    # MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Spectral contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spec_contrast_mean = np.mean(spec_contrast, axis=1)

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    tonnetz_mean = np.mean(tonnetz, axis=1)

    # ZCR
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))

    # RMS
    rms = np.mean(librosa.feature.rms(y=y))

    # Tempo
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    features = np.hstack([
        mfccs_mean,
        chroma_mean,
        spec_contrast_mean,
        tonnetz_mean,
        zcr,
        rms,
        tempo
    ])

    return features

# Lista di tutti i file .wav
feature_list = []
file_names = []
label = []

for file_name in os.listdir(AUDIO_DIR):
    if file_name.endswith(".wav"):
        file_path = os.path.join(AUDIO_DIR, file_name)
        try:
            features = extract_features(file_path)
            feature_list.append(features)
            label.append(LABEL)
            file_names.append(file_name)
        except Exception as e:
            print(f"Errore su {file_name}: {e}")

# Crea un DataFrame
feature_names = [f"mfcc{i}" for i in range(13)] + \
                [f"chroma{i}" for i in range(12)] + \
                [f"contrast{i}" for i in range(7)] + \
                [f"tonnetz{i}" for i in range(6)] + \
                ["zcr", "rms", "tempo"]

df = pd.DataFrame(feature_list, columns=feature_names)
df["label"] = label

# Normalizza se vuoi
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[feature_names]), columns=feature_names)
df_scaled["label"] = df["label"]

# Salva su file
df_scaled.to_csv(OUTPUT_NAME, index=False)

print("Feature extraction completata!")

Feature extraction completata!
